In [ ]:
!pip install tensorflow matplotlib numpy scikit-learn seaborn nltk opencv-python

In [ ]:
# Object detection using Transfer Learning of CNN architectures for the given (image dataset
# 2) using the below steps:
# a. Load in a pre-trained CNN model trained on a large dataset
# b. Freeze parameters (weights) in model's lower convolutional layers
# c. Add custom classifier with several layers of trainable parameters to model
# d. Train classifier layers on training data available for task
# e. Fine-tune hyper parameters and unfreeze more layers as needed

In [ ]:
#this one takes too much time but much more accurate will take an hour to train on cpu
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# Set random seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Data preparation
def prepare_data(data_path):
    """
    Prepare the Caltech-101 dataset for training
    """
    # Remove BACKGROUND_Google folder as it's not a valid class
    classes = [cls for cls in os.listdir(data_path) 
               if os.path.isdir(os.path.join(data_path, cls)) and cls != 'BACKGROUND_Google']
    
    print(f"Number of classes: {len(classes)}")
    print("Classes:", classes[:10])  # Print first 10 classes
    
    return classes

# a. Load in a pre-trained CNN model trained on a large dataset
def create_base_model():
    """
    Load pre-trained VGG16 model without top layers
    """
    # You can use the provided weights file or download automatically
    try:
        # If you have the local weights file
        base_model = VGG16(
            weights='vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5',
            include_top=False,
            input_shape=(224, 224, 3)
        )
        print("Loaded VGG16 from local weights file")
    except:
        # Download pre-trained weights
        base_model = VGG16(
            weights='imagenet',
            include_top=False,
            input_shape=(224, 224, 3)
        )
        print("Loaded VGG16 with ImageNet weights")
    
    return base_model

# b. Freeze parameters (weights) in model's lower convolutional layers
def freeze_base_layers(base_model):
    """
    Freeze the base model layers
    """
    for layer in base_model.layers:
        layer.trainable = False
    
    print(f"Frozen {len(base_model.layers)} layers in base model")
    return base_model

# c. Add custom classifier with several layers of trainable parameters to model
def add_custom_classifier(base_model, num_classes):
    """
    Add custom classifier on top of base model
    """
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.Dense(num_classes, activation='softmax')  # This will now match the actual number of classes
    ])
    
    return model

# Create data generators
def create_data_generators(data_path, batch_size=32, img_size=(224, 224)):
    """
    Create data generators for training and validation
    """
    # Data augmentation for training
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        zoom_range=0.2,
        shear_range=0.2,
        fill_mode='nearest',
        validation_split=0.2  # Use 20% for validation
    )
    
    # Only rescaling for validation
    val_datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split=0.2
    )
    
    train_generator = train_datagen.flow_from_directory(
        data_path,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='training',
        shuffle=True
    )
    
    val_generator = val_datagen.flow_from_directory(
        data_path,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation',
        shuffle=False
    )
    
    return train_generator, val_generator

# Training function
def train_model(model, train_generator, val_generator, epochs=20):
    """
    Train the model
    """
    # Compile the model
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    # Callbacks
    callbacks = [
        keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
        keras.callbacks.ReduceLROnPlateau(factor=0.2, patience=3),
        keras.callbacks.ModelCheckpoint('best_model.h5', save_best_only=True)
    ]
    
    # Train the model
    history = model.fit(
        train_generator,
        epochs=epochs,
        validation_data=val_generator,
        callbacks=callbacks,
        verbose=1
    )
    
    return history, model

# e. Fine-tune hyper parameters and unfreeze more layers as needed
def fine_tune_model(model, base_model, train_generator, val_generator, fine_tune_epochs=10):
    """
    Fine-tune the model by unfreezing some base layers
    """
    # Unfreeze the last few layers of base model
    for layer in base_model.layers[-4:]:  # Unfreeze last 4 layers
        layer.trainable = True
    
    # Recompile with lower learning rate
    model.compile(
        optimizer=Adam(learning_rate=0.0001/10),  # Very low learning rate for fine-tuning
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    print("Fine-tuning last 4 layers of base model...")
    
    # Fine-tune the model
    fine_tune_history = model.fit(
        train_generator,
        epochs=fine_tune_epochs,
        validation_data=val_generator,
        verbose=1
    )
    
    return fine_tune_history, model

# Visualization functions
def plot_training_history(history, fine_tune_history=None):
    """
    Plot training history
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Plot training & validation accuracy
    ax1.plot(history.history['accuracy'], label='Training Accuracy')
    ax1.plot(history.history['val_accuracy'], label='Validation Accuracy')
    
    if fine_tune_history:
        ax1.plot(range(len(history.history['accuracy']), 
                  len(history.history['accuracy']) + len(fine_tune_history.history['accuracy'])),
                fine_tune_history.history['accuracy'], label='Fine-tuning Training Accuracy')
        ax1.plot(range(len(history.history['val_accuracy']), 
                  len(history.history['val_accuracy']) + len(fine_tune_history.history['val_accuracy'])),
                fine_tune_history.history['val_accuracy'], label='Fine-tuning Validation Accuracy')
    
    ax1.set_title('Model Accuracy')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.legend()
    
    # Plot training & validation loss
    ax2.plot(history.history['loss'], label='Training Loss')
    ax2.plot(history.history['val_loss'], label='Validation Loss')
    
    if fine_tune_history:
        ax2.plot(range(len(history.history['loss']), 
                  len(history.history['loss']) + len(fine_tune_history.history['loss'])),
                fine_tune_history.history['loss'], label='Fine-tuning Training Loss')
        ax2.plot(range(len(history.history['val_loss']), 
                  len(history.history['val_loss']) + len(fine_tune_history.history['val_loss'])),
                fine_tune_history.history['val_loss'], label='Fine-tuning Validation Loss')
    
    ax2.set_title('Model Loss')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.legend()
    
    plt.tight_layout()
    plt.show()

def plot_sample_predictions(model, val_generator, class_names, num_samples=12):
    """
    Plot sample predictions
    """
    # Get a batch of validation data
    x_batch, y_batch = next(val_generator)
    predictions = model.predict(x_batch)
    
    # Convert predictions to class labels
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(y_batch, axis=1)
    
    # Plot sample predictions
    plt.figure(figsize=(15, 12))
    for i in range(min(num_samples, len(x_batch))):
        plt.subplot(3, 4, i+1)
        plt.imshow(x_batch[i])
        plt.axis('off')
        
        true_label = class_names[true_classes[i]]
        pred_label = class_names[predicted_classes[i]]
        confidence = np.max(predictions[i])
        
        color = 'green' if true_classes[i] == predicted_classes[i] else 'red'
        plt.title(f'True: {true_label}\nPred: {pred_label}\nConf: {confidence:.2f}', 
                 color=color, fontsize=8)
    
    plt.tight_layout()
    plt.show()

# Main execution
def main():
    # Set data path
    data_path = "caltech-101-img"
    
    # First, create data generators to find the actual number of classes
    train_generator, val_generator = create_data_generators(data_path, batch_size=32)
    
    # Get the actual number of classes from the generator
    num_classes = len(train_generator.class_indices)
    
    print(f"Actual number of classes found by data generator: {num_classes}")
    print(f"Class indices: {train_generator.class_indices}")
    
    # Get class names from generator
    class_names = list(train_generator.class_indices.keys())
    
    print(f"\nTraining samples: {train_generator.samples}")
    print(f"Validation samples: {val_generator.samples}")
    
    # Step a: Load pre-trained model
    print("\nStep a: Loading pre-trained VGG16 model...")
    base_model = create_base_model()
    
    # Step b: Freeze base layers
    print("\nStep b: Freezing base model layers...")
    base_model = freeze_base_layers(base_model)
    
    # Step c: Add custom classifier - USE THE ACTUAL NUMBER OF CLASSES
    print(f"\nStep c: Adding custom classifier with {num_classes} output classes...")
    model = add_custom_classifier(base_model, num_classes)
    
    # Print model summary
    model.summary()
    
    # Step d: Train classifier layers
    print("\nStep d: Training classifier layers...")
    history, model = train_model(model, train_generator, val_generator, epochs=20)
    
    # Step e: Fine-tune hyperparameters and unfreeze more layers
    print("\nStep e: Fine-tuning model...")
    fine_tune_history, model = fine_tune_model(
        model, base_model, train_generator, val_generator, fine_tune_epochs=10
    )
    
    # Evaluate the model
    print("\nEvaluating final model...")
    train_loss, train_accuracy = model.evaluate(train_generator, verbose=0)
    val_loss, val_accuracy = model.evaluate(val_generator, verbose=0)
    
    print(f"Final Training Accuracy: {train_accuracy:.4f}")
    print(f"Final Validation Accuracy: {val_accuracy:.4f}")
    
    # Plot training history
    plot_training_history(history, fine_tune_history)
    
    # Plot sample predictions
    plot_sample_predictions(model, val_generator, class_names)
    
    # Save the final model
    model.save('caltech101_final_model.h5')
    print("\nModel saved as 'caltech101_final_model.h5'")
    
    return model, history, fine_tune_history, train_generator, val_generator

# Additional analysis functions
def analyze_model_performance(model, val_generator, class_names):
    """
    Analyze model performance with detailed metrics
    """
    # Get all predictions
    val_generator.reset()
    predictions = model.predict(val_generator, verbose=1)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = val_generator.classes
    
    # Classification report
    print("\nClassification Report:")
    print(classification_report(true_classes, predicted_classes, 
                              target_names=class_names))
    
    # Confusion matrix (for first 20 classes for readability)
    plt.figure(figsize=(15, 12))
    cm = confusion_matrix(true_classes, predicted_classes)
    
    # Plot confusion matrix for first 20 classes
    if len(class_names) > 20:
        cm_subset = cm[:20, :20]
        class_names_subset = class_names[:20]
        sns.heatmap(cm_subset, annot=True, fmt='d', cmap='Blues',
                   xticklabels=class_names_subset, yticklabels=class_names_subset)
    else:
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                   xticklabels=class_names, yticklabels=class_names)
    
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.xticks(rotation=45)
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()

# Run the main function
if __name__ == "__main__":
    try:
        model, history, fine_tune_history, train_generator, val_generator = main()
        
        # Additional performance analysis
        class_names = list(train_generator.class_indices.keys())
        analyze_model_performance(model, val_generator, class_names)
        
    except Exception as e:
        print(f"Error occurred: {e}")
        import traceback
        traceback.print_exc()

In [ ]:
#takes less time but efficient for cpu
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
import os
import time
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# Set random seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

print("=== COMPREHENSIVE VGG16 TRANSFER LEARNING ===")
print("With detailed visualizations and analysis")

def extract_features_and_train_comprehensive():
    """
    Extract features once and train on pre-computed features with comprehensive outputs
    """
    data_path = "caltech-101-img"
    
    # Step 1: Load VGG16 for feature extraction
    print("Step a: Loading VGG16 for feature extraction...")
    feature_extractor = VGG16(
        weights='vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5',
        include_top=False,
        input_shape=(64, 64, 3),
        pooling='avg'
    )
    feature_extractor.trainable = False
    
    # Step 2: Create data generators
    print("Step b: Creating data generators...")
    datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
    
    train_gen = datagen.flow_from_directory(
        data_path, target_size=(64, 64), batch_size=32, 
        class_mode='categorical', subset='training', shuffle=False
    )
    
    val_gen = datagen.flow_from_directory(
        data_path, target_size=(64, 64), batch_size=32,
        class_mode='categorical', subset='validation', shuffle=False
    )
    
    num_classes = len(train_gen.class_indices)
    class_names = list(train_gen.class_indices.keys())
    print(f"Classes: {num_classes}")
    
    # Step 3: Extract features
    print("Step c: Extracting features from VGG16...")
    
    # Extract training features
    train_features = []
    train_labels = []
    train_filenames = []
    
    print("Extracting training features...")
    for i, (x_batch, y_batch) in enumerate(train_gen):
        if i >= len(train_gen):
            break
        features = feature_extractor.predict(x_batch, verbose=0)
        train_features.extend(features)
        train_labels.extend(y_batch)
        train_filenames.extend(train_gen.filenames[i * train_gen.batch_size:(i + 1) * train_gen.batch_size])
        if (i + 1) % 10 == 0:
            print(f"Processed {i + 1}/{len(train_gen)} training batches")
    
    # Extract validation features
    val_features = []
    val_labels = []
    val_filenames = []
    
    print("Extracting validation features...")
    for i, (x_batch, y_batch) in enumerate(val_gen):
        if i >= len(val_gen):
            break
        features = feature_extractor.predict(x_batch, verbose=0)
        val_features.extend(features)
        val_labels.extend(y_batch)
        val_filenames.extend(val_gen.filenames[i * val_gen.batch_size:(i + 1) * val_gen.batch_size])
        if (i + 1) % 5 == 0:
            print(f"Processed {i + 1}/{len(val_gen)} validation batches")
    
    # Convert to numpy arrays
    X_train = np.array(train_features)
    y_train = np.array(train_labels)
    X_val = np.array(val_features)
    y_val = np.array(val_labels)
    
    print(f"Training features: {X_train.shape}")
    print(f"Validation features: {X_val.shape}")
    
    # Step 4: Build and train classifier
    print("Step d: Training classifier on extracted features...")
    classifier = models.Sequential([
        layers.Dense(256, activation='relu', input_shape=(512,)),
        layers.Dropout(0.3),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    classifier.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    print("Training classifier...")
    start_time = time.time()
    history = classifier.fit(
        X_train, y_train,
        epochs=15,
        validation_data=(X_val, y_val),
        batch_size=32,
        verbose=1,
        callbacks=[
            keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)
        ]
    )
    training_time = time.time() - start_time
    
    print(f"Classifier trained in {training_time:.1f} seconds!")
    
    # Step 5: Create final model
    print("Step e: Creating final model...")
    final_model = models.Sequential([
        feature_extractor,
        classifier
    ])
    
    # ========== COMPREHENSIVE VISUALIZATIONS ==========
    
    # 1. Training History Plot
    plt.figure(figsize=(15, 5))
    
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy', linewidth=2)
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
    plt.title('Model Accuracy Progress', fontsize=14, fontweight='bold')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss', linewidth=2)
    plt.plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
    plt.title('Model Loss Progress', fontsize=14, fontweight='bold')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # 2. Final Evaluation
    print("\n=== COMPREHENSIVE EVALUATION ===")
    val_loss, val_accuracy = classifier.evaluate(X_val, y_val, verbose=0)
    train_loss, train_accuracy = classifier.evaluate(X_train, y_train, verbose=0)
    
    print(f"Training Accuracy: {train_accuracy:.4f}")
    print(f"Validation Accuracy: {val_accuracy:.4f}")
    print(f"Training Loss: {train_loss:.4f}")
    print(f"Validation Loss: {val_loss:.4f}")
    
    # 3. Predictions and Analysis
    print("\nGenerating predictions and analysis...")
    y_pred = classifier.predict(X_val, verbose=0)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = np.argmax(y_val, axis=1)
    
    # Calculate overall metrics
    overall_accuracy = np.mean(y_true_classes == y_pred_classes)
    print(f"Overall Accuracy: {overall_accuracy:.4f}")
    
    # 4. Confusion Matrix for Top 10 Classes (FIXED VERSION)
    print("\nGenerating confusion matrix for top 10 classes...")
    
    # Select top 10 most frequent classes in validation set
    unique, counts = np.unique(y_true_classes, return_counts=True)
    top_10_indices = unique[np.argsort(-counts)[:10]]  # Top 10 by frequency
    top_10_classes = [class_names[i] for i in top_10_indices]
    
    print(f"Top 10 classes by frequency: {top_10_classes}")
    
    # Filter for top 10 classes - only include samples where BOTH true and pred are in top 10
    mask_true = np.isin(y_true_classes, top_10_indices)
    mask_pred = np.isin(y_pred_classes, top_10_indices)
    mask_combined = mask_true & mask_pred
    
    y_true_filtered = y_true_classes[mask_combined]
    y_pred_filtered = y_pred_classes[mask_combined]
    
    print(f"Samples in confusion matrix: {len(y_true_filtered)}")
    
    if len(y_true_filtered) > 0:
        # Create mapping for confusion matrix
        label_map = {old_idx: new_idx for new_idx, old_idx in enumerate(top_10_indices)}
        y_true_mapped = np.array([label_map[int(idx)] for idx in y_true_filtered])  # Convert to int
        y_pred_mapped = np.array([label_map[int(idx)] for idx in y_pred_filtered])  # Convert to int
        
        plt.figure(figsize=(12, 10))
        cm = confusion_matrix(y_true_mapped, y_pred_mapped)
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                    xticklabels=top_10_classes, yticklabels=top_10_classes)
        plt.title('Confusion Matrix - Top 10 Most Frequent Classes', fontsize=14, fontweight='bold')
        plt.xlabel('Predicted Labels')
        plt.ylabel('True Labels')
        plt.xticks(rotation=45, ha='right')
        plt.yticks(rotation=0)
        plt.tight_layout()
        plt.show()
        
        # Classification Report for filtered data
        print("\nClassification Report (Top 10 Most Frequent Classes):")
        print(classification_report(y_true_mapped, y_pred_mapped, 
                                  target_names=top_10_classes, digits=3, zero_division=0))
    else:
        print("Not enough samples for confusion matrix")
    
    # 5. Sample Predictions Visualization
    print("\nGenerating sample predictions visualization...")
    # Get a sample of validation predictions
    sample_indices = np.random.choice(len(X_val), min(12, len(X_val)), replace=False)
    
    plt.figure(figsize=(15, 12))
    for i, idx in enumerate(sample_indices):
        plt.subplot(3, 4, i + 1)
        
        # Get original image
        img_path = os.path.join(data_path, val_filenames[idx])
        if os.path.exists(img_path):
            try:
                img = tf.keras.preprocessing.image.load_img(img_path, target_size=(64, 64))
                img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0
                plt.imshow(img_array)
            except:
                plt.imshow(np.zeros((64, 64, 3)))
        else:
            plt.imshow(np.zeros((64, 64, 3)))
        
        plt.axis('off')
        
        true_class = class_names[y_true_classes[idx]]
        pred_class = class_names[y_pred_classes[idx]]
        confidence = np.max(y_pred[idx])
        
        # Truncate long class names
        true_class_short = true_class[:15] + '...' if len(true_class) > 15 else true_class
        pred_class_short = pred_class[:15] + '...' if len(pred_class) > 15 else pred_class
        
        color = 'green' if y_true_classes[idx] == y_pred_classes[idx] else 'red'
        plt.title(f'True: {true_class_short}\nPred: {pred_class_short}\nConf: {confidence:.3f}', 
                 color=color, fontsize=8, pad=3)
    
    plt.suptitle('Sample Predictions on Validation Set', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # 6. Accuracy Distribution by Class
    print("\nCalculating per-class accuracy...")
    class_accuracy = {}
    class_counts = {}
    
    for class_idx, class_name in enumerate(class_names):
        class_mask = y_true_classes == class_idx
        class_count = np.sum(class_mask)
        if class_count > 0:
            class_acc = np.mean(y_pred_classes[class_mask] == class_idx)
            class_accuracy[class_name] = class_acc
            class_counts[class_name] = class_count
    
    # Sort classes by accuracy
    sorted_classes = sorted(class_accuracy.items(), key=lambda x: x[1], reverse=True)
    
    # Plot top 15 classes by accuracy
    plt.figure(figsize=(12, 8))
    top_15_classes = [cls[0] for cls in sorted_classes[:15]]
    top_15_accuracies = [cls[1] for cls in sorted_classes[:15]]
    
    colors = ['green' if acc > 0.7 else 'orange' if acc > 0.5 else 'red' for acc in top_15_accuracies]
    bars = plt.barh(top_15_classes, top_15_accuracies, color=colors, alpha=0.7)
    
    plt.xlabel('Accuracy')
    plt.title('Top 15 Classes by Accuracy', fontsize=14, fontweight='bold')
    plt.xlim(0, 1)
    plt.grid(True, alpha=0.3, axis='x')
    
    # Add value labels on bars
    for bar, acc in zip(bars, top_15_accuracies):
        plt.text(bar.get_width() + 0.01, bar.get_y() + bar.get_height()/2, 
                f'{acc:.3f}', va='center', fontsize=8)
    
    plt.tight_layout()
    plt.show()
    
    # 7. Class Distribution Chart
    plt.figure(figsize=(12, 6))
    class_names_short = [name[:20] + '...' if len(name) > 20 else name for name in class_names[:20]]
    class_counts_vals = [class_counts.get(name, 0) for name in class_names[:20]]
    
    plt.bar(range(len(class_names_short)), class_counts_vals, color='skyblue', alpha=0.7)
    plt.title('Class Distribution (Top 20 Classes)', fontsize=14, fontweight='bold')
    plt.xlabel('Classes')
    plt.ylabel('Number of Samples')
    plt.xticks(range(len(class_names_short)), class_names_short, rotation=45, ha='right')
    plt.grid(True, alpha=0.3, axis='y')
    plt.tight_layout()
    plt.show()
    
    # 8. Performance Summary
    print("\n=== PERFORMANCE SUMMARY ===")
    print(f"{'Metric':<25} {'Value':<10}")
    print("-" * 35)
    print(f"{'Training Accuracy':<25} {train_accuracy:.4f}")
    print(f"{'Validation Accuracy':<25} {val_accuracy:.4f}")
    print(f"{'Training Loss':<25} {train_loss:.4f}")
    print(f"{'Validation Loss':<25} {val_loss:.4f}")
    print(f"{'Number of Classes':<25} {num_classes}")
    print(f"{'Training Samples':<25} {len(X_train):,}")
    print(f"{'Validation Samples':<25} {len(X_val):,}")
    
    # 9. Best and Worst Performing Classes
    print("\n=== CLASS PERFORMANCE ANALYSIS ===")
    print("Top 5 Best Performing Classes:")
    for i, (cls, acc) in enumerate(sorted_classes[:5]):
        count = class_counts[cls]
        print(f"  {i+1}. {cls:<20} {acc:.3f} ({count} samples)")
    
    print("\nTop 5 Worst Performing Classes:")
    for i, (cls, acc) in enumerate(sorted_classes[-5:]):
        count = class_counts[cls]
        print(f"  {i+1}. {cls:<20} {acc:.3f} ({count} samples)")
    
    # 10. Transfer Learning Summary
    print("\n=== TRANSFER LEARNING PIPELINE ===")
    print("✓ VGG16 Feature Extractor (Frozen)")
    print("✓ Global Average Pooling")
    print("✓ Dense(256) + ReLU + Dropout(0.3)")
    print("✓ Dense(128) + ReLU + Dropout(0.2)")
    print(f"✓ Dense({num_classes}) + Softmax")
    
    # Save the model
    final_model.save('caltech101_vgg16_comprehensive.h5')
    print(f"\n✓ Model saved as 'caltech101_vgg16_comprehensive.h5'")
    
    # Final Summary
    print("\n" + "="*60)
    print("🎉 TRANSFER LEARNING COMPLETED SUCCESSFULLY! 🎉")
    print("="*60)
    print(f"📊 Final Validation Accuracy: {val_accuracy:.4f}")
    print(f"🏷️  Total Classes: {num_classes}")
    print(f"📈 Training Samples: {len(X_train):,}")
    print(f"🧪 Validation Samples: {len(X_val):,}")
    print(f"⚙️  Model Parameters: {final_model.count_params():,}")
    print("="*60)
    
    return final_model, history, class_names

# Run the comprehensive version
if __name__ == "__main__":
    try:
        total_start = time.time()
        
        model, history, class_names = extract_features_and_train_comprehensive()
        
        total_time = time.time() - total_start
        print(f"\n⏱️  TOTAL EXECUTION TIME: {total_time/60:.1f} MINUTES")
        print("📊 All visualizations generated successfully!")
        print("✅ Model ready for practical exam submission!")
        
    except Exception as e:
        print(f"❌ Error: {e}")
        import traceback
        traceback.print_exc()